In [1]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor
from sklearn.model_selection import ShuffleSplit
from mlxtend.regressor import StackingCVRegressor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
warnings.filterwarnings('ignore')

In [3]:
df_train = pd.read_csv('train.csv', index_col='id')

def listmerge(lstlst):
    all=[]
    for lst in lstlst:
        all.extend(lst)
    return all

t = pd.to_datetime(df_train.epoch)
df_train.epoch = (t-datetime.datetime(2013,1,1)).apply(
    lambda x: x.total_seconds()/2000)

for column in ['x_sim','y_sim', 'z_sim', 'Vx_sim','Vy_sim', 'Vz_sim']:
    col = []
    for id in df_train.sat_id.unique():
        lst = df_train[df_train['sat_id']==id][column]
        lst = np.diff(lst)
        col.append(np.insert(lst,0,0))
    col = listmerge(col)
    df_train[column+'_delta'] = col
    
for column in ['x_sim_delta','y_sim_delta', 'z_sim_delta', 'Vx_sim_delta','Vy_sim_delta', 'Vz_sim_delta']:
    for i in range(3):
        col = []
        for id in df_train.sat_id.unique():
            lst = df_train[df_train['sat_id']==id][column]
            lst = list(lst)
            for j in range(i+1):
                lst.pop()
                lst.insert(0, lst[0])
            col.append(lst)
        col_prev = col
        col = listmerge(col)
        df_train[column+'_prev'+str(i)] = col

for column in ['x_sim','y_sim', 'z_sim', 'Vx_sim','Vy_sim', 'Vz_sim']:
    for i in range(3):
        col = []
        for id in df_train.sat_id.unique():
            lst = df_train[df_train['sat_id']==id][column]
            lst = list(lst)
            for j in range(i+1):
                lst.pop()
                lst.insert(0, lst[0])
            col.append(lst)
        col_prev = col
        col = listmerge(col)
        df_train[column+'_prev'+str(i)] = col

col = []
for id in df_train.sat_id.unique():
    lst = df_train[df_train['sat_id']==id]['epoch']
    lst = np.diff(lst)
    col.append(np.insert(lst,0,0))
col = listmerge(col)
df_train['delta'] = col

df_train.head()  

,epoch,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,...,Vx_sim_prev0,Vx_sim_prev1,Vx_sim_prev2,Vy_sim_prev0,Vy_sim_prev1,Vy_sim_prev2,Vz_sim_prev0,Vz_sim_prev1,Vz_sim_prev2,delta
id,,,,,,,,,,,,,,,,,,,,,
0,15768.000000,0,-8855.823863,13117.780146,-20728.353233,-0.908303,-3.808436,-2.022083,-8843.131454,13138.221690,...,-0.907527,-0.907527,-0.907527,-3.804930,-3.804930,-3.804930,-2.024133,-2.024133,-2.024133,0.000000
1,15769.401500,0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,...,-0.907527,-0.907527,-0.907527,-3.804930,-3.804930,-3.804930,-2.024133,-2.024133,-2.024133,1.401500
2,15770.803001,0,-10578.684043,-10180.467460,-24238.280949,0.277435,-4.047522,0.723155,-10571.858472,-10145.939908,...,-0.303704,-0.907527,-0.907527,-4.269816,-3.804930,-3.804930,-0.616468,-2.024133,-2.024133,1.401501
3,15772.204500,0,-9148.251857,-20651.437460,-20720.381279,0.715600,-3.373762,1.722115,-9149.620794,-20618.200201,...,0.274880,-0.303704,-0.907527,-4.046788,-4.269816,-3.804930,0.718768,-0.616468,-2.024133,1.401500
4,15773.606001,0,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-6729.358857,-28902.271436,...,0.712437,0.274880,-0.303704,-3.375202,-4.046788,-4.269816,1.718306,0.718768,-0.616468,1.401501


In [4]:
df_test = pd.read_csv('test.csv', index_col='id')

t = pd.to_datetime(df_test.epoch)
df_test.epoch = (t-datetime.datetime(2013,1,1)).apply(
    lambda x: x.total_seconds()/2000)

for column in ['x_sim','y_sim', 'z_sim', 'Vx_sim','Vy_sim', 'Vz_sim']:
    col = []
    for id in df_test.sat_id.unique():
        lst = df_test[df_test['sat_id']==id][column]
        lst = np.diff(lst)
        last = df_test[df_test['sat_id']==id][column].iloc[0]-df_train[df_train['sat_id']==id][column].iloc[-1]
        col.append(np.insert(lst,0,last))
    col = listmerge(col)
    df_test[column+'_delta'] = col
    
for column in ['x_sim_delta','y_sim_delta', 'z_sim_delta', 'Vx_sim_delta','Vy_sim_delta', 'Vz_sim_delta']:
    for i in range(3):
        col = []
        for id in df_test.sat_id.unique():
            lst = df_test[df_test['sat_id']==id][column]
            lst = list(lst)
            for j in range(i+1):
                lst.pop()
                lst.insert(0, lst[0])
            col.append(lst)
        col_prev = col
        col = listmerge(col)
        df_test[column+'_prev'+str(i)] = col

for column in ['x_sim','y_sim', 'z_sim', 'Vx_sim','Vy_sim', 'Vz_sim']:
    for i in range(3):
        col = []
        for id in df_test.sat_id.unique():
            lst = df_test[df_test['sat_id']==id][column]
            lst = list(lst)
            for j in range(i+1):
                lst.pop()
                lst.insert(0, lst[0])
            col.append(lst)
        col_prev = col
        col = listmerge(col)
        df_test[column+'_prev'+str(i)] = col
    
col = []
for id in df_test.sat_id.unique():
    lst = df_test[df_test['sat_id']==id]['epoch']
    lst = np.diff(lst)
    col.append(np.insert(lst,0,0))
col = listmerge(col)
df_test['delta'] = col

df_test.head()  

,sat_id,epoch,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim,x_sim_delta,y_sim_delta,...,Vx_sim_prev0,Vx_sim_prev1,Vx_sim_prev2,Vy_sim_prev0,Vy_sim_prev1,Vy_sim_prev2,Vz_sim_prev0,Vz_sim_prev1,Vz_sim_prev2,delta
id,,,,,,,,,,,,,,,,,,,,,
3927,1,17107.252581,-13366.891347,-14236.753503,6386.774555,4.333815,-0.692764,0.810774,5015.921003,-1339.807567,...,4.333815,4.333815,4.333815,-0.692764,-0.692764,-0.692764,0.810774,0.810774,0.810774,0.000000
3928,1,17107.888503,-7370.434039,-14498.771520,7130.411325,5.077413,0.360609,0.313402,5996.457308,-262.018017,...,4.333815,4.333815,4.333815,-0.692764,-0.692764,-0.692764,0.810774,0.810774,0.810774,0.635922
3929,1,17108.524426,-572.068654,-13065.289498,7033.794876,5.519106,2.012830,-0.539412,6798.365385,1433.482022,...,5.077413,4.333815,4.333815,0.360609,-0.692764,-0.692764,0.313402,0.810774,0.810774,0.635923
3930,1,17109.160348,6208.945257,-9076.852425,5548.296900,4.849212,4.338955,-1.869600,6781.013911,3988.437074,...,5.519106,5.077413,4.333815,2.012830,0.360609,-0.692764,-0.539412,0.313402,0.810774,0.635922
3931,1,17109.796271,10768.200284,-2199.706707,2272.014862,1.940505,6.192887,-3.167724,4559.255027,6877.145718,...,4.849212,5.519106,5.077413,4.338955,2.012830,0.360609,-1.869600,-0.539412,0.313402,0.635923


In [5]:
x_cols = ['x_sim','y_sim','z_sim', 'Vx_sim','Vy_sim', 'Vz_sim']
y_cols = ['x','y','z', 'Vx','Vy', 'Vz']

In [6]:
cols_delta = []
cols = []
for column in ['x_sim_delta','y_sim_delta', 'z_sim_delta', 'Vx_sim_delta','Vy_sim_delta', 'Vz_sim_delta']:
    cols_delta.append([column+'_prev'+str(i) for i in range(3)])
    
for column in ['x_sim','y_sim', 'z_sim', 'Vx_sim','Vy_sim', 'Vz_sim']:
    cols.append([column+'_prev'+str(i) for i in range(3)])

In [7]:
def smape(satellite_predicted_values, satellite_true_values): 
    return np.mean(np.abs((satellite_predicted_values - satellite_true_values) 
        / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))

def score(satellite_predicted_values, satellite_true_values):
    return 100*(1-np.mean(smape(satellite_predicted_values, satellite_true_values)))

In [17]:
sats = []

id_list = df_test.sat_id.unique()
for id in range(len(id_list)):
    print(id, end=', ')
    models = []
    for i in range(len(y_cols)):
        x = df_train[df_train['sat_id']==id_list[id]][['epoch', 'delta', x_cols[i], x_cols[i]+'_delta']+cols[i]+cols_delta[i]].values
        y = df_train[df_train['sat_id']==id_list[id]][[y_cols[i]]].values

        et = ExtraTreesRegressor()
        stack = et

        stack.fit(x, y)
        models.append(stack)
    sats.append(models)

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 

# Test sample

In [19]:
df_result = pd.DataFrame()
y_cols_test = ['sat_id','x','y','z', 'Vx','Vy', 'Vz']
x_cols_test = ['sat_id', 'x_sim','y_sim','z_sim', 'Vx_sim','Vy_sim', 'Vz_sim']
df_result[y_cols_test] = df_test[x_cols_test]

full_results = [] 
id_list = df_test.sat_id.unique()
for id in range(len(id_list)):
    results = [] 
    for i in range(len(y_cols)):
        x = df_test[df_test['sat_id']==id_list[id]]
        x = x[['epoch', 'delta', x_cols[i], x_cols[i]+'_delta']+cols[i]+cols_delta[i]]
        df = pd.DataFrame()

        model = sats[id][i]
        preds = model.predict(x)
        results.append(preds)
    full_results.append(results)
    
for id in range(len(id_list)):
    ids = df_test[df_test['sat_id']==id_list[id]].index
    for i in range(len(y_cols)):
        new_series = pd.Series(full_results[id][i])
        new_series.index = ids
        df_result[y_cols[i]][ids] = new_series
df_result = df_result.drop('sat_id', axis=1)
df_result.to_csv('submission.csv',index=True)
df_result.head()

,x,y,z,Vx,Vy,Vz
id,,,,,,
3927,-16237.973619,-13285.824416,7614.313664,4.508792,-0.643170,0.887486
3928,-15234.741943,-13322.013444,8207.469344,4.341312,-0.382195,0.624707
3929,-9645.788025,-13372.610163,8779.095963,4.637067,-0.473527,0.517675
3930,-11168.468393,-13651.180539,9234.817375,4.592359,0.274917,0.142853
3931,-4886.513948,-12787.533894,8973.617893,5.262422,1.301646,-0.648646


In [13]:
df_test.head()

,sat_id,epoch,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim,x_sim_delta,y_sim_delta,...,Vx_sim_prev0,Vx_sim_prev1,Vx_sim_prev2,Vy_sim_prev0,Vy_sim_prev1,Vy_sim_prev2,Vz_sim_prev0,Vz_sim_prev1,Vz_sim_prev2,delta
id,,,,,,,,,,,,,,,,,,,,,
3927,1,17107.252581,-13366.891347,-14236.753503,6386.774555,4.333815,-0.692764,0.810774,5015.921003,-1339.807567,...,4.333815,4.333815,4.333815,-0.692764,-0.692764,-0.692764,0.810774,0.810774,0.810774,0.000000
3928,1,17107.888503,-7370.434039,-14498.771520,7130.411325,5.077413,0.360609,0.313402,5996.457308,-262.018017,...,4.333815,4.333815,4.333815,-0.692764,-0.692764,-0.692764,0.810774,0.810774,0.810774,0.635922
3929,1,17108.524426,-572.068654,-13065.289498,7033.794876,5.519106,2.012830,-0.539412,6798.365385,1433.482022,...,5.077413,4.333815,4.333815,0.360609,-0.692764,-0.692764,0.313402,0.810774,0.810774,0.635923
3930,1,17109.160348,6208.945257,-9076.852425,5548.296900,4.849212,4.338955,-1.869600,6781.013911,3988.437074,...,5.519106,5.077413,4.333815,2.012830,0.360609,-0.692764,-0.539412,0.313402,0.810774,0.635922
3931,1,17109.796271,10768.200284,-2199.706707,2272.014862,1.940505,6.192887,-3.167724,4559.255027,6877.145718,...,4.849212,5.519106,5.077413,4.338955,2.012830,0.360609,-1.869600,-0.539412,0.313402,0.635923
